# Part 1

## Chapter 1.4

### Section 1.4.1

In [1]:
import pandas as pd
from scipy import stats
import joblib
import numpy as np
from collections import namedtuple

In [2]:
df = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [3]:
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [4]:
# Implement (4)
df_male = df.copy()[df["segment"] != "Womens E-Mail"]
df_male["treatment"] = (df_male["segment"].values == "Mens E-Mail").astype(int)

In [5]:
df_male.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0,0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0,1
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0.0,1
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0.0,1
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0.0,0


In [6]:
# TEST (4)
assert type(df_male) is pd.DataFrame  # 女性向けメールが配信されなかったグループのみ含むdf
assert "Womens E-Mail" not in df_male["segment"]  # 女性向けメールが配信されていない
assert set(df_male[df_male["treatment"] == 1]["segment"]) == set(["Mens E-Mail"])  # treatmentには男性向けメールが配信されたグループのみ
assert set(df_male.keys()) == set(df.keys()).union(set(["treatment"]))

### Section 1.4.2

In [7]:
# Implement (5)
df_aggregated = df_male.groupby('treatment').agg(
    conversion_rate=pd.NamedAgg('conversion', 'mean'),
    spend_mean=pd.NamedAgg('spend', 'mean'),
    count=pd.NamedAgg('treatment', 'count'),
)

In [8]:
df_aggregated

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


In [9]:
# TEST (5)
assert type(df_aggregated) is pd.DataFrame  # 集計されたdf
assert set(["conversion_rate", "spend_mean", "count"]).issubset(df_aggregated.keys())  # 指定のカラムを持つ
assert (df_aggregated.sort_values("treatment")["conversion_rate"].values - np.array([0.005726, 0.01253])).sum() < 1e-4
assert (df_aggregated.sort_values("treatment")["spend_mean"].values - np.array([0.652789, 1.422617])).sum() < 1e-4
assert (df_aggregated.sort_values("treatment")["count"].values - np.array([21306, 21307])).sum() < 1e-4

In [10]:
# Implement (6)
ttest_result = stats.ttest_ind(
    df_male["spend"][df_male["treatment"] == 1],
    df_male["spend"][df_male["treatment"] == 0]
)
pvalue = float(ttest_result.pvalue)

In [11]:
pvalue

1.163200872605869e-07

In [12]:
# TEST (6)
assert type(pvalue) is float  # result of stats.ttest_ind
assert pvalue < 0.05  # 有意

### Section 1.4.3

In [13]:
# Implement (7) - No tests. これ以降の乱数の影響をなくすため
bias = (df_male.history > 300) | (df_male.recency < 6) | (df_male.channel=='Multichannel')
df_biased = pd.concat([
    df_male[(df_male["treatment"] == 0) & bias].sample(frac=0.5, random_state=1),
    df_male[(df_male["treatment"] == 0) & ~bias],
    df_male[(df_male["treatment"] == 1) & bias],
    df_male[(df_male["treatment"] == 1) & ~bias].sample(frac=0.5, random_state=1),
], ignore_index=True)

In [14]:
df_biased.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,8,5) $500 - $750,572.65,1,0,Urban,1,Web,No E-Mail,0,0,0.0,0
1,5,1) $0 - $100,42.38,1,0,Urban,1,Phone,No E-Mail,1,0,0.0,0
2,1,"7) $1,000 +",3003.48,1,1,Urban,1,Phone,No E-Mail,0,0,0.0,0
3,1,5) $500 - $750,662.10,0,1,Urban,1,Web,No E-Mail,0,0,0.0,0
4,5,1) $0 - $100,44.37,0,1,Urban,0,Web,No E-Mail,0,0,0.0,0


In [15]:
# Implement (8)
df_aggregated_biased = df_biased.groupby('treatment').agg(
    conversion_rate=pd.NamedAgg('conversion', 'mean'),
    spend_mean=pd.NamedAgg('spend', 'mean'),
    count=pd.NamedAgg('treatment', 'count'),
)

In [16]:
df_aggregated_biased

,conversion_rate,spend_mean,count
treatment,,,
0,0.004540,0.557954,14757
1,0.013572,1.541704,17168


In [17]:
# TEST (8)
assert type(df_aggregated_biased) is pd.DataFrame
assert (
    df_aggregated_biased.sort_values("treatment")["conversion_rate"][0]
    < df_aggregated.sort_values("treatment")["conversion_rate"][0]
), "conversion_rate drops in controll due to bias"
assert (
    df_aggregated_biased.sort_values("treatment")["conversion_rate"][1]
    > df_aggregated.sort_values("treatment")["conversion_rate"][1]
), "conversion_rate increase in treatment due to bias"
assert (
    df_aggregated_biased.sort_values("treatment")["spend_mean"][1]
    > df_aggregated.sort_values("treatment")["spend_mean"][1]
), "spend_mean increase in treatment due to bias"

In [18]:
# Implement (9)
ttest_result_biased = stats.ttest_ind(
    df_biased.spend[df_biased["treatment"] == 1],
    df_biased.spend[df_biased["treatment"] == 0]
)
pvalue_biased = float(ttest_result_biased.pvalue)

In [19]:
pvalue_biased

2.21319841336543e-08

In [20]:
# TEST (9)
assert type(pvalue_biased) is float
assert pvalue_biased < 0.05  # バイアスがあるが有意という結果が出るはず"
assert pvalue_biased < ttest_result.pvalue  # バイアスのせいでp値が小さくなるはず

# Part 2

## Chapter 2.1

### Section 2.1.1

In [21]:
X_biased = df_biased[["treatment", "history"]]
y_biased = df_biased["spend"]

In [22]:
# Implement (6)
import statsmodels.api as sm

model_biased = sm.OLS(y_biased, sm.add_constant(X_biased))  # Adding intercept
result_biased = model_biased.fit()
beta_treatment_biased = float(result_biased.params[result_biased.params.index == "treatment"])
pvalue_treatment_biased = float(result_biased.pvalues[result_biased.params.index == "treatment"])
beta_history_biased = float(result_biased.params[result_biased.params.index == "history"])
pvalue_history_biased = float(result_biased.pvalues[result_biased.params.index == "history"])

/usr/local/var/pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/statsmodels/compat/pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)
/usr/local/var/pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [23]:
result_biased.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     20.45
Date:                Tue, 30 Jun 2020   Prob (F-statistic):           1.32e-09
Time:                        21:47:21   Log-Likelihood:            -1.3312e+05
No. Observations:               31925   AIC:                         2.663e+05
Df Residuals:                   31922   BIC:                         2.663e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3413      0.147      2.327      0.020       0.054       0.629
treatment      0.9088      0.177      5.122      0.000       0.561       1.257
history        0.0011      0.000      3.096      0.002       0.000       0.002
==============================================================================
Omnibus:                    70760.532   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        352134568.791
Skew:                          20.807   Prob(JB):                         0.00
Kurtosis:                     515.825   Cond. No.                         833.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# TEST (6)
assert type(beta_treatment_biased) is float
assert type(pvalue_treatment_biased) is float
assert type(beta_history_biased) is float
assert type(pvalue_history_biased) is float
assert abs(beta_treatment_biased - 0.9026) < 1e-2
assert pvalue_treatment_biased < 0.05  # 有意
assert abs(beta_history_biased - 0.0010) < 1e-2

## Chapter 2.2

### Section 2.2.1

In [25]:
X = df_male[["treatment"]]
y = df_male["spend"]

X_biased = df_biased[["treatment"]]
y_biased = df_biased["spend"]

X_biased_mreg = df_biased[["treatment", "recency", "channel", "history"]].copy()
X_biased_mreg["channel"] = (X_biased_mreg["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_mreg = df_biased["spend"]

In [26]:
# Implement (7)
import statsmodels.api as sm

model = sm.OLS(y, sm.add_constant(X))  # Adding intercept
result = model.fit()
beta_treatment = float(result.params[result.params.index == "treatment"])
pvalue_treatment = float(result.pvalues[result.params.index == "treatment"])

model_biased = sm.OLS(y_biased, sm.add_constant(X_biased))  # Adding intercept
result_biased = model_biased.fit()
beta_treatment_biased = float(result_biased.params[result_biased.params.index == "treatment"])
pvalue_treatment_biased = float(result_biased.pvalues[result_biased.params.index == "treatment"])

model_biased_mreg = sm.OLS(y_biased_mreg, sm.add_constant(X_biased_mreg))  # Adding intercept
result_biased_mreg = model_biased_mreg.fit()
beta_treatment_biased_mreg = float(result_biased_mreg.params[result_biased_mreg.params.index == "treatment"])
pvalue_treatment_biased_mreg = float(result_biased_mreg.pvalues[result_biased_mreg.params.index == "treatment"])

In [27]:
beta_treatment, beta_treatment_biased, beta_treatment_biased_mreg

(0.7698271558945372, 0.9837495599336821, 0.8618682527375361)

In [28]:
# TEST (7)
assert type(beta_treatment) is float
assert type(pvalue_treatment) is float
assert type(beta_treatment_biased) is float
assert type(pvalue_treatment_biased) is float
assert type(beta_treatment_biased_mreg) is float
assert type(pvalue_treatment_biased_mreg) is float

assert abs(beta_treatment - 0.770) < 1e-2
gain_from_df_aggregated = df_aggregated["spend_mean"][1] - df_aggregated["spend_mean"][0]
assert abs(beta_treatment - gain_from_df_aggregated) < 1e-8  # 1章の結果と同じになるはず
assert pvalue_treatment < 0.05  # 有意
assert abs(beta_treatment_biased - 0.979) < 1e-2  # バイアスによりbeta_treatmentより大きくなる
assert pvalue_treatment_biased < 0.05  # 有意
assert beta_treatment_biased_mreg < beta_treatment_biased  # 共変量を加えてバイアスの影響が小さくなる

### Section 2.2.3

In [29]:
X_biased_omit = df_biased[["treatment", "recency", "channel"]].copy()  # historyを抜いたdf
X_biased_omit["channel"] = (X_biased_omit["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_omit = df_biased["spend"]

X_biased = df_biased[["treatment", "recency", "channel", "history"]].copy()
X_biased["channel"] = (X_biased["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased = df_biased["spend"]

X_biased_history = df_biased[["treatment", "recency", "channel"]].copy()
X_biased_history["channel"] = (X_biased_history["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_history = df_biased["history"]

In [30]:
# Implement (9)

import statsmodels.api as sm

model = sm.OLS(y_biased_omit, sm.add_constant(X_biased_omit))  # Adding intercept
result = model.fit()
alpha_treatment = float(result.params[result.params.index == "treatment"])

model = sm.OLS(y_biased, sm.add_constant(X_biased))  # Adding intercept
result = model.fit()
beta_treatment = float(result.params[result.params.index == "treatment"])
beta_history = float(result.params[result.params.index == "history"])

model = sm.OLS(y_biased_history, sm.add_constant(X_biased_history))  # Adding intercept
result = model.fit()
gamma_treatment = float(result.params[result.params.index == "treatment"])

In [31]:
beta_history * gamma_treatment, (alpha_treatment - beta_treatment)

(0.04300977779866467, 0.0430097777986469)

In [32]:
# TEST (9)
assert type(alpha_treatment) is float  # モデルAにおけるtreatmentの係数
assert type(beta_treatment) is float  # モデルBにおけるtreatmentの係数
assert type(beta_history) is float  # モデルBにおけるhistoryの係数
assert type(gamma_treatment) is float  # モデルCにおけるtreatmentの係数

assert abs(beta_history * gamma_treatment - (alpha_treatment - beta_treatment)) < 1e-8  # 知られている式

### Section 2.2.7

In [33]:
X_biased = df_biased[["treatment", "recency", "channel", "history"]].copy()
X_biased["channel"] = (X_biased["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased = df_biased["spend"]

X_biased_visit = df_biased[["treatment", "recency", "channel", "history", "visit"]].copy()
X_biased_visit["channel"] = (X_biased_visit["channel"] == "Web").astype(int)  # 2種しかないカテゴリカル変数なのでバイナリに変換
y_biased_visit = df_biased["spend"]

In [34]:
# Implement (10)
r, p = stats.pearsonr(X_biased_visit["visit"], y_biased_visit)
cor_spend_visit, pvalue_spend_visit = float(r), float(p)

model = sm.OLS(y_biased, sm.add_constant(X_biased))  # Adding intercept
result = model.fit()
beta_treatment_biased = float(result.params[result.params.index == "treatment"])

model = sm.OLS(y_biased_visit, sm.add_constant(X_biased_visit))  # Adding intercept
result = model.fit()
beta_treatment_biased_visit = float(result.params[result.params.index == "treatment"])

In [35]:
cor_spend_visit, pvalue_spend_visit

(0.16595931934315053, 6.706529778181228e-196)

In [36]:
beta_treatment_biased, beta_treatment_biased_visit

(0.8618682527375361, 0.27602780187309517)

In [37]:
# TEST (10)
assert type(cor_spend_visit) is float  # spendとvisitの相関係数
assert type(pvalue_spend_visit) is float  # spendとvisitの相関のp値
assert type(beta_treatment_biased) is float  # treatmentの効果
assert type(beta_treatment_biased_visit) is float  # treatmentの効果(visitが変数に入る場合)

assert abs(cor_spend_visit - 0.144) < 5e-2
assert pvalue_spend_visit < 0.05  # 有意

assert beta_treatment_biased > beta_treatment_biased_visit  # visitのせいでtreatmentの介入効果が低く見える

In [38]:
cor_spend_visit, pvalue_spend_visit

(0.16595931934315053, 6.706529778181228e-196)